In [1]:
from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2

# from spektral.layers import GraphConv
# GRaphConv is deprecated, use GCNConv or GCSConv instead
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.utils import normalized_laplacian
from spektral.layers import GCSConv  # as GraphConv
from spektral.layers import GINConv # as GraphConv
from spektral.layers import GCNConv  # as GraphConv

from spektral.utils.convolution import gcn_filter  # For GCNConv
from spektral.utils.convolution import normalized_adjacency  # For GCSConv

import numpy as np
import scipy.sparse
import pandas as pd
import pathlib
import os
import json

from sklearn.model_selection import train_test_split

In [2]:
# Parameters
l2_reg = 5e-4  # Regularization rate for l2
learning_rate = 1e-3  # Learning rate for SGD
batch_size = 32  # Batch size
epochs = 5  # Number of training epochs
es_patience = 200  # Patience fot early stopping

In [3]:
current_path = pathlib.Path().absolute()
path = current_path.parent

In [4]:
# load a npz file
# the npz file is a sparse matrix
# as all the matrices are equal, we can use the first one is equal to all the others

adj_angry_path = path / 'angry_adj'
print(adj_angry_path)
for file in os.listdir(adj_angry_path)[:1]:
    file_path = adj_angry_path / file
    sparse_matrix = scipy.sparse.load_npz(file_path)
    sparse_matrix = sparse_matrix.todense()


c:\Users\rodri\Documents\Rodrigo\Insper\SextoSemestre\Facial-Emotion-Classification-Graph_Fork\angry_adj


In [5]:
angry_path = path / 'angry_meshpoints'
disgusted_path = path / 'disgusted_meshpoints'
happy_path = path / 'happy_meshpoints'
neutral_path = path / 'neutral_meshpoints'
sad_path = path / 'sad_meshpoints'
surprised_path = path / 'surprised_meshpoints'

path_list = [angry_path, disgusted_path, happy_path, neutral_path, sad_path, surprised_path]

In [6]:
def extrai_dist(path_list, dists_list, limit=0.1):
    for path in path_list:
        quant_files = len(os.listdir(path))
        count = 0
        for file in os.listdir(path):
            if (count / quant_files) > limit:
                break
            file_path = path / file
            count += 1
            with open(file_path, 'r') as f:
                data = json.load(f)
                dists_list[path_list.index(path)].append(data)

In [7]:
dists_surprised = []
dists_disgusted = []
dists_happy = []
dists_neutral = []
dists_sad = []
dists_angry = []

dists_list = [dists_angry, dists_disgusted, dists_happy, dists_neutral, dists_sad, dists_surprised]

extrai_dist(path_list, dists_list, 0.1)

print(len(dists_angry))
print(len(dists_disgusted))
print(len(dists_happy))
print(len(dists_neutral))
print(len(dists_sad))
print(len(dists_surprised))

446
161
810
687
742
402


In [8]:
# dists_list

In [9]:
# import concat from tensorflow
from tensorflow import concat
tensor_mesh = concat(dists_list, axis=0)

In [10]:
tensor_mesh.shape

TensorShape([3248, 478, 2])

In [11]:
# target list
target_list = []
for i in range(6):
    target_list.append(np.full(len(dists_list[i]), i))

target_list = concat(target_list, axis=0)


In [12]:
target_list.shape

TensorShape([3248])

In [13]:
sparse_matrix.shape

(468, 468)

In [14]:
n_out = 6  # Number of classes
N = sparse_matrix.shape[0]  # Number of nodes in the graphs
F = tensor_mesh.shape[-1]  # Original feature dimensionality

# Até aqui está exatamente como o professor sugeriu

## um tensor com 3 dimensões: (número de amostras, número de vértices, número de atributos)
## Outro com o target: (classe de cada amostra)

# GINConv Model

https://graphneural.network/layers/convolution/#ginconv

In [15]:
X_in = Input(shape=(N, F))
A_in = Input(tensor=sp_matrix_to_sp_tensor(sparse_matrix)) # A as a fixed tensor, otherwise Keras will complain about inputs of different rank.

In [16]:
graph_conv_1 = GINConv(32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True)([X_in, A_in])
graph_conv_2 = GINConv(32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True)([graph_conv_1, A_in])
flatten = Flatten()(graph_conv_2)
fc = Dense(512, activation="relu")(flatten)
output = Dense(n_out, activation="softmax")(fc)

In [17]:
# Build model
model = Model(inputs=[X_in, A_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["acc"]
)

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 468, 2)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(468, 468)]                 0         []                            
                                                                                                  
 gin_conv (GINConv)          (None, 468, 32)              97        ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 gin_conv_1 (GINConv)        (None, 468, 32)              1057      ['gin_conv[0][0]',        

# Still need to figure out how to use the GINConv model

In [86]:
# o tensor_mesh é um tensor de dimensão 3
# o sparse_matrix é um tensor de dimensão 2
# o target_list é um tensor de dimensão 1

# Train model
validation_data = ([tensor_mesh, sparse_matrix], target_list)
model.fit(
    [tensor_mesh, sparse_matrix],
    target_list,
    batch_size=batch_size,
    validation_data=validation_data,
    shuffle=True,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=es_patience, restore_best_weights=True)],
)


ValueError: Data cardinality is ambiguous:
  x sizes: 3248, 468
  y sizes: 3248
Make sure all arrays contain the same number of samples.